In [1]:
# importing necessary libraries
import numpy as np
import scipy.stats as spstats
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import LabelEncoder , OneHotEncoder


%matplotlib inline
# matplotlib
mpl.style.reload_library()
mpl.style.use('classic')
mpl.rcParams['figure.facecolor'] = (1, 1, 1, 0)
mpl.rcParams['figure.figsize'] = [6.0, 4.0]
mpl.rcParams['figure.dpi'] = 100

In [2]:
# Reading in the dataset
poke_df = pd.read_csv("pokemon.csv")
poke_df.head()

,id,species,generation_id,height,weight,base_experience,type_1,type_2,hp,attack,defense,speed,special-attack,special-defense
0,1,bulbasaur,1,0.7,6.9,64,grass,poison,45,49,49,45,65,65
1,2,ivysaur,1,1.0,13.0,142,grass,poison,60,62,63,60,80,80
2,3,venusaur,1,2.0,100.0,236,grass,poison,80,82,83,80,100,100
3,4,charmander,1,0.6,8.5,62,fire,NaN,39,52,43,65,60,50
4,5,charmeleon,1,1.1,19.0,142,fire,NaN,58,64,58,80,80,65


In [3]:
poke_df[["hp","attack","defense"]].describe()

,hp,attack,defense
count,807.000000,807.000000,807.000000
mean,68.748451,76.086741,71.726146
std,26.032808,29.544598,29.730228
min,1.000000,5.000000,5.000000
25%,50.000000,55.000000,50.000000
50%,65.000000,75.000000,67.000000
75%,80.000000,95.000000,89.000000
max,255.000000,181.000000,230.000000


In [4]:
atk_def = poke_df[['attack', 'defense']]

In [5]:
pf = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
res = pf.fit_transform(atk_def)
res

array([[   49.,    49.,  2401.,  2401.,  2401.],
       [   62.,    63.,  3844.,  3906.,  3969.],
       [   82.,    83.,  6724.,  6806.,  6889.],
       ...,
       [  131.,   211., 17161., 27641., 44521.],
       [  127.,    53., 16129.,  6731.,  2809.],
       [  112.,    75., 12544.,  8400.,  5625.]])

In [6]:
pd.unique(poke_df["generation_id"])

array([1, 2, 3, 4, 5, 6, 7])

In [7]:
# gen_le = LabelEncoder()
# genre_label = gen_le.fit_transform(vg_df["Genre"])
# vg_df["Genre_label"] = genre_label

In [8]:
gen_ohe = OneHotEncoder()
gen_feature_arr = gen_ohe.fit_transform(poke_df[['gen_Label']]).toarray()
gen_feature_labels = list(gen_le.classes_)
gen_features = pd.DataFrame(gen_feature_arr, columns=gen_feature_labels)
# encode legendary status labels using one-hot encoding scheme

KeyError: "None of [Index(['gen_Label'], dtype='object')] are in the [columns]"

In [ ]:
vg_df = pd.read_csv("vgsales.csv", encoding="utf-8")
vg_df[['Name', 'Platform', 'Year', 'Genre', 'Publisher']].iloc[1:7]

In [ ]:
genres = pd.unique(vg_df["Genre"])
genres

In [ ]:
gle = LabelEncoder()
gle.fit_transform(vg_df["Genre"])
genre_labels = {index: label for index, label in enumerate(gle.classes_)}
genre_labels

In [ ]:
import numpy as np
import pandas as pd
import re
import nltk

In [ ]:
corpus = ['The sky is blue and beautiful.',
             'Love this blue and beautiful sky!',
             'The quick brown fox jumps over the lazy dog.',
             'The brown fox is quick and the blue dog is lazy!',
             'The sky is very blue and the sky is very beautiful today',
             'The dog is lazy but the brown fox is quick!']

labels = ['weather', 'weather', 'animals', 'animals', 'weather', 'animals']

corpus  = np.array(corpus)

In [ ]:
corpus_df = pd.DataFrame({'Document': corpus,
                         "Category": labels})
corpus_df = corpus_df[["Document", "Category"]]
corpus_df

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words("english")
def normalize_document(doc):
       # lower case and remove special characters\whitespaces
       doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I)
       doc = doc.lower()
       doc = doc.strip()
       # tokenize document
       tokens = wpt.tokenize(doc)
       # filter stopwords out of document
       filtered_tokens = [token for token in tokens if token not in stop_words]
       # re-create document from filtered tokens
       doc = ' '.join(filtered_tokens)
       return doc
    
    
normalize_corpus = np.vectorize(normalize_document)
norm_corpus = normalize_corpus(corpus)
norm_corpus

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(min_df = 0., max_df = 1.)

cv_matrix = cv.fit_transform(norm_corpus)
cv_matrix = cv_matrix.toarray()
cv_matrix

In [ ]:
vocab = cv.get_feature_names()
pd.DataFrame(cv_matrix, columns=vocab)

In [ ]:
bv = CountVectorizer(ngram_range=(2,2))
bv_matrix = bv.fit_transform(norm_corpus)
bv_matrix = bv_matrix.toarray()
bv_matrix

In [ ]:
vocab = bv.get_feature_names()
pd.DataFrame(bv_matrix, columns=vocab)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer()
tv_matrix = tv.fit_transform(norm_corpus)
tv_matrix = tv_matrix.toarray()
vocab = tv.get_feature_names()
pd.DataFrame(tv_matrix, columns=vocab)